# Talk Recommender - Pycon 2018

With 32 tuotorials, 12 sponsor workshops, 16 talks at the education summit, and 95 talks at the main conference - Pycon has a lot to offer. Reading through all the talk descriptions and filtering out the ones that you should go to is a tedious process. But not anymore.

## Introducing TalkRecommender
Talk recommender is a recommendation system that recommends talks from this year's Pycon based on the ones that you went to last year.  This way you don't waste any time preparing a schedule and get to see the talks that matter the most to you! 

As shown in the demo, the users are asked to label previous year's talks into two categories - the one that they went to in person, and the ones they watched later online. Talk Recommender uses those labels to predict talks from this year that will be interesing to them. 

We will be using [`pandas`](https://pandas.pydata.org/) abd [`scikit-learn`](http://scikit-learn.org/) to build and the model.

*Remember to click on Save and Checkpoint from the File menu to save changes you made to the notebook* 

### Exercise A: Load the data
The data directory contains the snapshot of one such user's labeling - lets load that up and start with our analysis. 

In [1]:
!ls -lrt data

total 184
-rw-r--r-- 1 1000 1000 186903 Apr 26 13:02 talks.csv


In [2]:
import pandas as pd
import numpy as np
df=pd.read_csv('data/talks.csv')
df.head()

,id,title,description,presenters,date_created,date_modified,location,talk_dt,year,label
0,1,5 ways to deploy your Python web app in 2017,You’ve built a fine Python web application and...,Andrew T. Baker,2018-04-19 00:59:20.151875,2018-04-19 00:59:20.151875,Portland Ballroom 252–253,2017-05-08 15:15:00.000000,2017,0.0
1,2,A gentle introduction to deep learning with Te...,Deep learning's explosion of spectacular resul...,Michelle Fullwood,2018-04-19 00:59:20.158338,2018-04-19 00:59:20.158338,Oregon Ballroom 203–204,2017-05-08 16:15:00.000000,2017,0.0
2,3,aiosmtpd - A better asyncio based SMTP server,smtpd.py has been in the standard library for ...,Barry Warsaw,2018-04-19 00:59:20.161866,2018-04-19 00:59:20.161866,Oregon Ballroom 203–204,2017-05-08 14:30:00.000000,2017,1.0
3,4,Algorithmic Music Generation,Music is mainly an artistic act of inspired cr...,Padmaja V Bhagwat,2018-04-19 00:59:20.165526,2018-04-19 00:59:20.165526,Portland Ballroom 251 & 258,2017-05-08 17:10:00.000000,2017,0.0
4,5,An Introduction to Reinforcement Learning,Reinforcement learning (RL) is a subfield of m...,Jessica Forde,2018-04-19 00:59:20.169075,2018-04-19 00:59:20.169075,Portland Ballroom 252–253,2017-05-08 13:40:00.000000,2017,0.0


Here is a brief description of the interesting fields.

variable | description  
------|------|
`title`|Title of the talk
`description`|Description of the talk
`year`|Is it a `2017` talk or `2018`  
`label`|`1` indicates the user preferred seeing the talk in person,<br> `0` indicates they would schedule it for later.

Note all 2018 talks are set to 1. However they are only placeholders, and are not used in training the model. We will  use only 2017 data for training.

Lets start by selecting the 2017 talk descriptions that were labeled by the user for watching in person.

```python
df[(df.year==2017) & (df.label==1)]['description']
```

Print the description of the talks that the user preferred watching in person. How many such talks are there?

### Exercise B: Feature Extraction
In this step we build the feature set by tokenization, counting and normalization of the bi-grams from the text descriptions of the talk. You can find more information on text feature extraction [here](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction) and TfidfVectorizer [here](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words="english")

The 2017 talks will be used for training and the 2018 talks will we used for predicting. Set the values of `year_labeled` and `year_predict` to appropriate values.

In [4]:
year_labeled=2017
year_predict=2018
vectorized_text_labeled = vectorizer.fit_transform(df[df.year==year_labeled]['description'])
vectorized_text_predict = vectorizer.transform(df[df.year==year_predict]['description'])

### Exercise C: Split into Training and Testing Set

Next we split our data into training set and testing set. This allows us to do cross validation and avoid overfitting. Use the `train_test_split` method from `sklearn.model_selection` to split the `vectorized_text_labeled` into training and testing set with the test size as one third of the size (0.3) of the labeled.

[Here](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) is the documentation for the function.

In [5]:
from sklearn.model_selection import train_test_split
labels = df[df.year == 2017]['label']
test_size=0.3
X_train, X_test, y_train, y_test = train_test_split(vectorized_text_labeled, labels, test_size=test_size, random_state=1)

### Exercise D: Train the model
Finally we get to the stage for training the model. We are going to use a linear [support vector classifier](http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html) and check its accuracy by using the `classification_report` function. Note that we have not done any parameter tuning yet, so your model might not give you the best results. 


[Here](http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html) is some information for using [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) for doing exhaustive search over specified parameter values of an estimator. _However, this is purely for reference and not needed for this exercise._

Print out the `report` to see how well your model has been trained!

In [6]:
import sklearn
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
classifier = LinearSVC()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
report = sklearn.metrics.classification_report(y_test, y_pred)

### Exercise E: Make Predictions
Use the model to predict which 2018 talks the user should go to. 

Using the `predicted_talk_indexes` print out the talk id, description, presenters, title and location and talk date.
How many talks should the user go to according to your model?

In [7]:
predicted_talks_vector = classifier.predict(vectorized_text_predict)
df_2018 = df[df.year==2018]

# Offset the rows by 2017 talks
predicted_talk_indexes = predicted_talks_vector.nonzero()[0] + len(df[df.year==2017])
df_2018.loc[predicted_talk_indexes][['id','description','presenters','title','location','talk_dt']]
# your solution goes here

,id,description,presenters,title,location,talk_dt
102,103,"Nowadays, there are many ways of building data...",Christopher Fonnesbeck,Bayesian Non-parametric Models for Data Scienc...,Global Center Ballroom AB,2018-03-29 13:40:00.000000
103,104,Behavior-Driven Development (BDD) is gaining p...,Andrew Knight,Behavior-Driven Python,Grand Ballroom A,2018-03-29 12:10:00.000000
105,106,"You've used pytest and you've used mypy, but b...",Hillel Wayne,Beyond Unit Tests: Taking Your Testing to the ...,Room 26A/B/C,2018-03-29 12:10:00.000000
112,113,Want to know about the latest trends in the Py...,"Dmitry Filippov, Ewa Jodlowska",By the Numbers: Python Community Trends in 201...,Room 26A/B/C,2018-03-29 13:55:00.000000
113,114,Python now offers static types! Companies like...,Greg Price,Clearer Code at Scale: Static Types at Zulip a...,Grand Ballroom B,2018-03-29 13:50:00.000000
126,126,One of the most challenging and important thin...,Lisa Roach,Demystifying the Patch Function,Grand Ballroom B,2018-03-29 12:10:00.000000
131,132,Are you an intermediate python developer looki...,Nina Zakharenko,Elegant Solutions For Everyday Python Problems,Room 26A/B/C,2018-03-29 17:10:00.000000
134,135,"Today, services built on Python 3.6.3 are wide...",Jason Fried,Fighting the Good Fight: Python 3 in your orga...,Grand Ballroom B,2018-03-29 16:30:00.000000
138,139,"During peak hours, Netflix video streams make ...",Amjith Ramanujam,How Netflix does failovers in 7 minutes flat,Global Center Ballroom AB,2018-03-29 11:30:00.000000
139,140,A function is a small chunk of code that does ...,Jack Diederich,HOWTO Write a Function,Room 26A/B/C,2018-03-29 12:10:00.000000


### Exercise F: Expose it as a service

Now that you have pieces of the code ready, copy them together into the `model.py` file located in this folder, and rebuild your docker image. Copy the code from the above cells into the body of the `prediction` function.

Lets rebuild the docker image and start an new container following the comments.

In the following steps you will leave the jupyter notebook, and stop the container serving it. So save any changes you have done till this point.

```
docker stop <container_name>
docker build -t recommender .
docker run -p 8888:8888 -p 9000:9000 -v $(pwd):/app recommender
```
where `<container_name>` is the name of the container serving this jupyter notebook.

The `api.py` file in this directory is a flask app that makes call to the `model.py` module and exposes the model built in the previous steps as a service. In order to start the flask server, open a new terminal and run the following command.

```
docker exec $(docker ps -ql) python api.py
```
Where `docker ps -ql` gets numeric id of the latest container id.

Finally go to http://0.0.0.0:9000/predict to see the talks that were recommended for this user.

In [1]:
## http://0.0.0.0:9000/predict?id=103 for filtering

### Exercise G: Pickle the model

Finally we do not have to retrain our model anytime we have to make predictions. In most real life data science applications, the training phase is a time consuming proecss. We would seaprately train and serialize the model which is then exposed through the api to make the predictions. The `predict_api` directory of the TalkVoter app shows an approach where we wrap the model and seaprate out only calls to the prediction api to use the trained model instead of reprocessing any time there is a call to the api.

In [8]:
from sklearn.externals import joblib
with open('talk_recommender.pkl', 'wb') as f:
    joblib.dump(classifier, f)

This will create the pickle file in your directory.

!ls -l

Use the `joblib.load` function to read the `classifier` back from the `talk_recommender.pkl` file.

In [9]:
!ls -l

total 116
-rw-r--r-- 1 1000 1000   283 Apr 26 17:29 Dockerfile
-rw-r--r-- 1 1000 1000   400 Apr 26 13:02 README.md
drwxr-xr-x 2 root root  4096 Apr 26 18:33 __pycache__
-rw-r--r-- 1 1000 1000   298 Apr 26 13:02 api.py
drwxr-xr-x 2 1000 1000  4096 Apr 26 17:56 data
-rw-r--r-- 1 1000 1000   670 Apr 26 13:02 model.py
-rw-r--r-- 1 1000 1000   167 Apr 26 13:02 requirements.txt
-rw-r--r-- 1 1000 1000 17908 Apr 26 18:39 talk_recommender.ipynb
-rw-r--r-- 1 root root 66755 Apr 26 18:40 talk_recommender.pkl


In [10]:
classifier=joblib.load('talk_recommender.pkl')